In [20]:
import pandas as pd
import hyperloglog
import copy

In [77]:
queries_sets_df = pd.read_pickle("./queries_sets.pkl")
songs_sets_df = pd.read_pickle("./songs_sets.pkl")

In [78]:
cols = queries_sets_df.columns
set_cols = list(filter(lambda x: x.endswith("_set"), cols))
set_cols

['pitch_diff-ngram_5_set',
 'UpSameDown-ngram_5_set',
 'pitch_diff-ngram_8_set',
 'UpSameDown-ngram_8_set',
 'pitch_diff-ngram_10_set',
 'UpSameDown-ngram_10_set',
 'pitch_diff-ngram_12_set',
 'UpSameDown-ngram_12_set',
 'pitch_diff-multi_gram_5_10_set',
 'UpSameDown-multi_gram_5_10_set']

In [79]:
def create_hll_from_set(l, error_rate):
    hll = hyperloglog.HyperLogLog(error_rate = error_rate)
    for item in l:
        hll.add(item)
    return hll

In [80]:
hll_config_list = [0.26, 0.065, 0.018, 0.005]
hll_p_list = []

In [81]:
for error_rate in hll_config_list:
    p = hyperloglog.HyperLogLog(error_rate).p
    hll_p_list.append(p)
    for set_col in set_cols:
        hll_col = f"{set_col}_hll_{p}"
        queries_sets_df[hll_col] = queries_sets_df[set_col].apply(lambda x: create_hll_from_set(x,error_rate))
        songs_sets_df[hll_col] = songs_sets_df[set_col].apply(lambda x: create_hll_from_set(x,error_rate))

In [82]:
queries_sets_df.head(2)

,Query ID,Song ID,pitch_diff-ngram_5_set,UpSameDown-ngram_5_set,pitch_diff-ngram_8_set,UpSameDown-ngram_8_set,pitch_diff-ngram_10_set,UpSameDown-ngram_10_set,pitch_diff-ngram_12_set,UpSameDown-ngram_12_set,...,pitch_diff-ngram_5_set_hll_16,UpSameDown-ngram_5_set_hll_16,pitch_diff-ngram_8_set_hll_16,UpSameDown-ngram_8_set_hll_16,pitch_diff-ngram_10_set_hll_16,UpSameDown-ngram_10_set_hll_16,pitch_diff-ngram_12_set_hll_16,UpSameDown-ngram_12_set_hll_16,pitch_diff-multi_gram_5_10_set_hll_16,UpSameDown-multi_gram_5_10_set_hll_16
0,q1,1118,"{(-1, 0, -2, -2, -3), (-2, 5, -1, 0, -2), (-3,...","{('D', 'U', 'D', 'S', 'D'), ('D', 'D', 'D', 'S...","{(-2, 5, -1, 0, -2, -2, -3, 0), (2, 0, -2, 5, ...","{('S', 'D', 'D', 'D', 'S', 'U', 'U', 'U'), ('S...","{(-2, 5, -1, 0, -2, -2, -3, 0, 3, 2), (2, 0, -...","{('D', 'U', 'D', 'S', 'D', 'D', 'D', 'S', 'U',...","{(0, -2, 5, -1, 0, -2, -2, -3, 0, 3, 2, 3), (2...","{('S', 'D', 'U', 'D', 'S', 'D', 'D', 'D', 'S',...",...,<hyperloglog.hll.HyperLogLog object at 0x3696d...,<hyperloglog.hll.HyperLogLog object at 0x36974...,<hyperloglog.hll.HyperLogLog object at 0x36974...,<hyperloglog.hll.HyperLogLog object at 0x36971...,<hyperloglog.hll.HyperLogLog object at 0x3697b...,<hyperloglog.hll.HyperLogLog object at 0x36976...,<hyperloglog.hll.HyperLogLog object at 0x369b4...,<hyperloglog.hll.HyperLogLog object at 0x369b3...,<hyperloglog.hll.HyperLogLog object at 0x1775e...,<hyperloglog.hll.HyperLogLog object at 0x15fcf...
1,q2,1438,"{(-1, -2, 10, 2, -2), (2, -2, -4, -2, 2), (1, ...","{('D', 'U', 'U', 'D', 'D'), ('D', 'D', 'D', 'D...","{(-4, -2, 2, 6, -6, -2, -2, -2), (2, -2, -4, -...","{('D', 'U', 'U', 'D', 'D', 'D', 'U', 'U'), ('U...","{(-2, -4, -2, 2, 6, -6, -2, -2, -2, 0), (1, -1...","{('U', 'U', 'D', 'D', 'D', 'U', 'U', 'D', 'D',...","{(-2, -4, -2, 2, 6, -6, -2, -2, -2, 0, 2, -2),...","{('D', 'U', 'U', 'D', 'D', 'D', 'U', 'U', 'D',...",...,<hyperloglog.hll.HyperLogLog object at 0x15fcf...,<hyperloglog.hll.HyperLogLog object at 0x36971...,<hyperloglog.hll.HyperLogLog object at 0x3697b...,<hyperloglog.hll.HyperLogLog object at 0x36976...,<hyperloglog.hll.HyperLogLog object at 0x369b4...,<hyperloglog.hll.HyperLogLog object at 0x369b3...,<hyperloglog.hll.HyperLogLog object at 0x369ba...,<hyperloglog.hll.HyperLogLog object at 0x15fcf...,<hyperloglog.hll.HyperLogLog object at 0x1751c...,<hyperloglog.hll.HyperLogLog object at 0x369a2...


In [83]:
songs_sets_df.head(2)

,Song ID,pitch_diff-ngram_5_set,UpSameDown-ngram_5_set,pitch_diff-ngram_8_set,UpSameDown-ngram_8_set,pitch_diff-ngram_10_set,UpSameDown-ngram_10_set,pitch_diff-ngram_12_set,UpSameDown-ngram_12_set,pitch_diff-multi_gram_5_10_set,...,pitch_diff-ngram_5_set_hll_16,UpSameDown-ngram_5_set_hll_16,pitch_diff-ngram_8_set_hll_16,UpSameDown-ngram_8_set_hll_16,pitch_diff-ngram_10_set_hll_16,UpSameDown-ngram_10_set_hll_16,pitch_diff-ngram_12_set_hll_16,UpSameDown-ngram_12_set_hll_16,pitch_diff-multi_gram_5_10_set_hll_16,UpSameDown-multi_gram_5_10_set_hll_16
0,1,"{(2, 0, 1, -1, 1), (0, 1, 6, -3, 0), (1, 12, -...","{('D', 'U', 'U', 'D', 'D'), ('U', 'S', 'U', 'D...","{(2, -16, -4, 11, -11, 0, 11, 5), (-1, 1, 0, -...","{('D', 'D', 'U', 'D', 'U', 'D', 'S', 'D'), ('U...","{(6, 0, -2, 3, -3, 2, -1, -1, 2, -2), (9, -12,...","{('D', 'D', 'U', 'D', 'U', 'U', 'D', 'U', 'S',...","{(3, 0, -1, 2, 0, -1, 3, -1, 2, -2, 0, -1), (-...","{('U', 'S', 'D', 'U', 'S', 'D', 'D', 'U', 'D',...","{(-2, 1, 1, -1, 1, 0, -2), (-3, 2, -1, -1, 2),...",...,<hyperloglog.hll.HyperLogLog object at 0x17567...,<hyperloglog.hll.HyperLogLog object at 0x15fcf...,<hyperloglog.hll.HyperLogLog object at 0x3697c...,<hyperloglog.hll.HyperLogLog object at 0x36974...,<hyperloglog.hll.HyperLogLog object at 0x3697b...,<hyperloglog.hll.HyperLogLog object at 0x3697f...,<hyperloglog.hll.HyperLogLog object at 0x369b8...,<hyperloglog.hll.HyperLogLog object at 0x369bb...,<hyperloglog.hll.HyperLogLog object at 0x369bb...,<hyperloglog.hll.HyperLogLog object at 0x176bd...
1,4,"{(24, -24, -9, 9, -15), (0, 4, 0, 2, 0), (16, ...","{('D', 'U', 'U', 'D', 'D'), ('U', 'S', 'U', 'D...","{(8, -8, 8, -8, 2, 17, 0, -19), (0, -5, -14, 0...","{('D', 'U', 'D', 'S', 'U', 'D', 'U', 'S'), ('D...","{(28, -24, 24, -24, -9, 9, -15, 15, -16, 7), (...","{('D', 'U', 'U', 'S', 'D', 'S', 'D', 'S', 'U',...","{(-2, 0, -1, 0, 0, 0, 1, -1, 0, 0, 5, -33), (0...","{('U', 'D', 'S', 'S', 'S', 'S', 'D', 'U', 'U',...","{(-35, 7, 9, 19, 0, -5, -14, 0, -16), (24, -24...",...,<hyperloglog.hll.HyperLogLog object at 0x17567...,<hyperloglog.hll.HyperLogLog object at 0x36974...,<hyperloglog.hll.HyperLogLog object at 0x3697b...,<hyperloglog.hll.HyperLogLog object at 0x3697f...,<hyperloglog.hll.HyperLogLog object at 0x369b8...,<hyperloglog.hll.HyperLogLog object at 0x369ba...,<hyperloglog.hll.HyperLogLog object at 0x369bb...,<hyperloglog.hll.HyperLogLog object at 0x369bb...,<hyperloglog.hll.HyperLogLog object at 0x17568...,<hyperloglog.hll.HyperLogLog object at 0x1776e...


In [84]:
non_vocals_song_ids = [1081,126,1212, 1536, 1655, 995,1786, 814]

In [85]:
queries_sets_df = queries_sets_df[~queries_sets_df["Song ID"].isin(non_vocals_song_ids)]
songs_sets_df = songs_sets_df[~songs_sets_df["Song ID"].isin(non_vocals_song_ids)]

In [87]:
queries_sets_df.shape

(112, 52)

In [88]:
songs_sets_df.shape

(473, 51)

In [86]:
queries_sets_df.to_pickle("./queries_set_hll.pkl")
songs_sets_df.to_pickle("./songs_set_hll.pkl")

# Calculate Similarities

In [10]:
 cross_df = queries_sets_df[["Query ID"]].merge(songs_sets_df[["Song ID"]], how="cross")

In [14]:
cross_df

,Query ID,Song ID
0,q1,1
1,q1,4
2,q1,7
3,q1,14
4,q1,15
...,...,...
56753,q118,2419
56754,q118,2421
56755,q118,2424
56756,q118,2429


In [12]:
def get_overlap_coef(a,b):
    inter_set = a.intersection(b)
    den = min(len(a),len(b))
    if(den == 0):
        return -1
    return len(inter_set)/den

def get_set_similarity(query_id, song_id, set_col):
    query_set = queries_sets_df[queries_sets_df["Query ID"]==query_id].iloc[0][set_col]
    song_set = songs_sets_df[songs_sets_df["Song ID"]==song_id].iloc[0][set_col]
    return get_overlap_coef(query_set, song_set)

In [15]:
# SIMILARITY CALCULATE BY SETS
for set_col in set_cols:
    sim_col = f"similarity_{set_col}_baseline"
    cross_df[sim_col] = cross_df.apply(lambda x: get_set_similarity(x["Query ID"],x["Song ID"], set_col), axis=1)
    cross_df.to_pickle(f"./checkpoint/{set_col}.pkl")

In [31]:
# HLL++ FUNCTIONS
def get_merged_hll(hll1, hll2):
    m_hll = copy.copy(hll1)
    m_hll.update(hll2)
    return m_hll

def estimate_overlapping_coef(hll1: hyperloglog.HyperLogLog, hll2:hyperloglog.HyperLogLog):
    card_1 = hll1.card()
    card_2 = hll2.card()
    m_hll = get_merged_hll(hll1, hll2)
    union_card = m_hll.card()
    dem = min(card_1,card_2)
    if(dem == 0):
        return -1
    return  (card_1+card_2-union_card)/dem
def get_set_similarity_by_hll(query_id, song_id, set_col, hll_p):
    hll_col = f"{set_col}_hll_{hll_p}"
    query_hll = queries_sets_df[queries_sets_df["Query ID"]==query_id].iloc[0][hll_col]
    song_hll = songs_sets_df[songs_sets_df["Song ID"]==song_id].iloc[0][hll_col]
    return estimate_overlapping_coef(query_hll, song_hll)

In [32]:
# SIMILARITY BY HLL
for set_col in set_cols:
    for hll_p in hll_p_list:
        sim_col = f"similarity_{set_col}_hll_{hll_p}"
        cross_df[sim_col] = cross_df.apply(lambda x: get_set_similarity_by_hll(x["Query ID"],x["Song ID"], set_col, hll_p), axis=1)
        cross_df.to_pickle(f"./checkpoint/{sim_col}.pkl")

IOStream.flush timed out


In [57]:
cross_df = pd.read_pickle("./checkpoint/similarity_UpSameDown-multi_gram_5_10_set_hll_16.pkl")

In [58]:
cross_df = cross_df[~cross_df["Song ID"].isin(non_vocals_song_ids)]
queries_sets_df = queries_sets_df[~queries_sets_df["Song ID"].isin(non_vocals_song_ids)]

In [59]:
# compute index for each query
def get_index_of_song_in_query(query_id,song_id, col_sim):
    query_results = cross_df[cross_df["Query ID"] == query_id]
    query_results = query_results.sort_values(by=[col_sim], ascending=False)
    list_results = query_results["Song ID"].tolist()
    if(song_id in list_results):
        return list_results.index(song_id)+1
    else:
        return None

In [60]:
queries_indexes_df = queries_sets_df[["Query ID","Song ID"]].copy()
for set_col in set_cols:
    col_index = f"{set_col}_index"
    sim_col = f"similarity_{set_col}_baseline"
    queries_indexes_df[col_index] = queries_indexes_df.apply(lambda x:get_index_of_song_in_query(x["Query ID"],x["Song ID"],sim_col), axis=1)        

In [61]:
for set_col in set_cols:
    for hll_p in hll_p_list:
        col_index = f"{set_col}_hll_{hll_p}_index"
        sim_col = f"similarity_{set_col}_hll_{hll_p}"
        queries_indexes_df[col_index] = queries_indexes_df.apply(lambda x:get_index_of_song_in_query(x["Query ID"],x["Song ID"],sim_col), axis=1)

In [65]:
queries_indexes_df

,Query ID,Song ID,pitch_diff-ngram_5_set_index,UpSameDown-ngram_5_set_index,pitch_diff-ngram_8_set_index,UpSameDown-ngram_8_set_index,pitch_diff-ngram_10_set_index,UpSameDown-ngram_10_set_index,pitch_diff-ngram_12_set_index,UpSameDown-ngram_12_set_index,...,UpSameDown-ngram_12_set_hll_12_index,UpSameDown-ngram_12_set_hll_16_index,pitch_diff-multi_gram_5_10_set_hll_4_index,pitch_diff-multi_gram_5_10_set_hll_8_index,pitch_diff-multi_gram_5_10_set_hll_12_index,pitch_diff-multi_gram_5_10_set_hll_16_index,UpSameDown-multi_gram_5_10_set_hll_4_index,UpSameDown-multi_gram_5_10_set_hll_8_index,UpSameDown-multi_gram_5_10_set_hll_12_index,UpSameDown-multi_gram_5_10_set_hll_16_index
0,q1,1118,400,427,398,406,398,398,398,398,...,164,127,439,266,210,141,115,151,341,450
1,q2,1438,14,113,60,102,60,179,60,113,...,432,389,26,331,289,390,399,166,133,100
2,q3,1546,62,10,7,33,7,166,7,29,...,468,62,3,270,446,14,85,8,17,10
3,q4,1579,17,71,32,76,30,98,30,81,...,307,113,183,416,332,269,396,17,63,63
4,q5,1808,1,146,193,204,193,307,193,207,...,331,299,153,101,38,1,415,100,143,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,q114,479,1,69,283,21,283,45,283,257,...,8,410,23,77,39,6,140,77,16,29
114,q115,1747,211,93,205,61,205,180,205,250,...,205,432,181,198,52,139,145,269,108,67
115,q116,2024,228,460,210,314,210,185,210,245,...,307,105,195,250,328,381,90,423,444,443
116,q117,281,335,329,333,396,333,333,333,333,...,333,333,337,383,432,168,443,469,432,453


In [63]:
queries_indexes_df.to_pickle("./queries_indexes_df.pkl")

In [64]:
# queries_indexes_df.sort_values(by='pitch_diff-ngram_5_set_index').tail(31)

,Query ID,Song ID,pitch_diff-ngram_5_set_index,UpSameDown-ngram_5_set_index,pitch_diff-ngram_8_set_index,UpSameDown-ngram_8_set_index,pitch_diff-ngram_10_set_index,UpSameDown-ngram_10_set_index,pitch_diff-ngram_12_set_index,UpSameDown-ngram_12_set_index,...,UpSameDown-ngram_12_set_hll_12_index,UpSameDown-ngram_12_set_hll_16_index,pitch_diff-multi_gram_5_10_set_hll_4_index,pitch_diff-multi_gram_5_10_set_hll_8_index,pitch_diff-multi_gram_5_10_set_hll_12_index,pitch_diff-multi_gram_5_10_set_hll_16_index,UpSameDown-multi_gram_5_10_set_hll_4_index,UpSameDown-multi_gram_5_10_set_hll_8_index,UpSameDown-multi_gram_5_10_set_hll_12_index,UpSameDown-multi_gram_5_10_set_hll_16_index
111,q112,2026,243,203,235,221,235,135,235,243,...,359,184,39,72,444,353,330,92,303,242
62,q63,391,258,336,251,212,251,260,251,240,...,145,408,464,171,136,52,372,415,198,219
89,q90,1175,262,116,238,297,237,375,237,280,...,295,444,435,151,259,247,464,457,233,255
95,q96,597,275,375,270,427,270,442,270,302,...,138,190,407,79,410,277,241,462,407,413
5,q6,2133,276,211,144,179,144,220,144,156,...,471,466,107,456,265,411,312,223,174,171
110,q111,2113,282,348,194,335,211,241,211,186,...,187,340,102,229,291,459,440,249,393,385
74,q75,479,293,104,283,181,283,128,283,272,...,146,35,452,399,115,240,3,88,131,152
16,q17,96,313,161,308,219,308,332,308,311,...,30,386,355,140,75,34,283,192,253,207
77,q78,70,320,36,314,18,314,50,314,320,...,422,239,168,59,142,229,294,70,15,17
19,q20,60,321,140,318,39,318,42,318,319,...,449,344,392,126,163,386,188,49,21,41
